# LLama.CPP
LLama.CPP is a tool that enables us to run LLM inference on a local machine. This is true for both desktop and mobile (which is our main focus). \
In this notebook I present results of my research on this tool; how we can use, where we can use it, and what are its limitations.

# 1. Build Llama.cpp on Desktop
For obvious reasons (such as physical keyboard, mouse etc), experimenting with Llama.cpp is much easier on desktop. \
So while we target mobile platforms, for learning and research purposes I worked with the desktop version, \
and so here are the steps to run Llama.cpp on desktop (here I present steps for Linux, but Windows version is also available in the repository docs):
1. Clone the Llama.cpp repository, which can be found here: https://github.com/ggerganov/llama.cpp
2. Install CMake (if you don't have it already) by running `sudo apt install cmake`
3. Install Clang compiler by running `sudo apt install clang`
4. (Optional) On mobile i install libllvm instead (it should contain clang and cmake). I didn't try this, but you can.
5. Enter the llama.cpp directory and create a build directory by running `cmake -B build`
6. Build the project by running `cmake --build build --config Release`


# 2. Run LLM on Desktop
There are actually three ways to run LLMs (not true for BERTs, I will describe this later). For each of them,
you will need a model saved in a .gguf format (which is a serialized model format used by Llama.cpp). \
TheBloke user on Hugging Face has shared a lot of models converted to this format; you can find him here: https://huggingface.co/TheBloke.
For the purpose of examples here, I will use TheBloke/Llama-2-7B-GGUF (https://huggingface.co/TheBloke/Llama-2-7B-GGUF).

## 1. Run LLM directly from the command line
First of all, you need to download the model. When you hed to the model page and its files, it will contain \
a lot of different versions with different quantizations (described on the model's page). \
I will use the Q5 version. Now, you can download and store it anywhere, but for performance reasons \
(at least on android) it is suggested to store it under ~/ path.\
Now, you can run the model by running the following command:
```bash
./build/bin/llama-simple -m ~/llama-2-7b.Q5_K_S.gguf -c 4096 -p "What is the capital of France?"
```
Now, it is a big model, and running it on CPU may take a while. But, if everything goes smoothly, you should \
get something similar to this: \
![llama-7B](./ResearchImages/llama7bdesktop.png)

## 2. Run LLM as a server
Llama.cpp has a basic server for running with LLMs (and it actually has a nice API, so we can write scripts to communicate with it). \
To run the server, you need to run the following command:
```bash
./build/bin/llama-server -m ~/llama-2-7b.Q5_K_S.gguf -c 4096
```
After that you can go under http://127.0.0.1:8080/ and you should see the server's page: \
![llama-server](./ResearchImages/LLamacppserver.png)
In this particular case, results are very bad. But it is dependent on the model, I played with smaller ones \
and while responses were not perfect, they were making sense and were actually related to the topic.


In [ ]:
# Take Termux from f-droid

# pkg install cmake
# pkg install libllvm
# git clone llamacpp
# cd llamacpp
# cmake -B build
# cmake --build build --config Release

# cd ..
# curl -L -o llama-2-7b-chat.Q2_K.gguf https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q2_K.gguf
# cd llamacpp
# ./build/bin/llama-server -m ~/tinyllama-1.1b-chat-v0.3.Q8_0.gguf -c 4096


# Works for both mobile and desktop
# https://github.com/ggerganov/llama.cpp/tree/master/examples/server has a server with API,
# so technically we can make app, deploy server somewhere on mobile and app would call the local server

# has some android demo to checkout, do it

https://www.reddit.com/r/LocalLLaMA/comments/14rncnb/local_llama_on_android_phone/
